# Set the namespaces based on _your_ deployment

In [3]:
# Paste the namespace of your flow here. You can find it in Metaflow GUI. 
# For the baseline.
# something like: 'production:mfprj-xsfdb3gtsiboqyrd-0-vqsy'
champ_namespace = "production:mfprj-xsfdb3gtsiboqyrd-0-vqsy" 
print(f"namespace = {champ_namespace}")

namespace = production:mfprj-xsfdb3gtsiboqyrd-0-vqsy


# Configure the endpoints to set model state on the server

In [4]:
endpoint_uri_base = 'http://127.0.0.1:8000/'
slug = 'load-models?champ_namespace={}'.format(champ_namespace)
load_models_endpoint = endpoint_uri_base + slug
print(load_models_endpoint)

http://127.0.0.1:8000/load-models?champ_namespace=production:mfprj-xsfdb3gtsiboqyrd-0-vqsy


# Run the uvicorn server

Open a terminal and run:
```sh
uvicorn model-server:api
```
This needs to stay running while you complete the remaining cells of the notebook.

In [19]:
# TEST Server
import requests
try:
    print(f"URI = {endpoint_uri_base}")
    response = requests.get(endpoint_uri_base, verify=False, proxies={"https": endpoint_uri_base})
    print(f"Reponse = {response.json()}")
except requests.ConnectionError as e: 
    print("Did you run the server?!")

URI = http://127.0.0.1:8000/
Reponse = {'message': 'Hello there from model-server.py!'}


# Load the champion model state on the server

In [20]:
import requests
try:
    print(f"URI = {load_models_endpoint}")
    response = requests.get(load_models_endpoint, verify=False, proxies={"https": endpoint_uri_base})
    print(response.json())
except requests.ConnectionError as e: 
    print("Did you run the server?!")

URI = http://127.0.0.1:8000/load-models?champ_namespace=production:mfprj-xsfdb3gtsiboqyrd-0-vqsy
Running baseline model as champion.


# Get sample data

In [16]:
# get random data point
import pandas as pd
data = pd.read_csv("../data/titanic.csv")

inference_example = data.sample()
idx = inference_example.index[0]
true_target = inference_example['Survived'].values[0]
print(inference_example.head())
print(f"target = {true_target}")

     PassengerId  Survived  Pclass                        Name     Sex  Age  \
274          275         1       3  Healy, Miss. Hanora "Nora"  female  NaN   

     SibSp  Parch  Ticket  Fare Cabin Embarked  
274      0      0  370375  7.75   NaN        Q  
target = 1


In [17]:
payload = inference_example.to_json()
print(payload)

{"PassengerId":{"274":275},"Survived":{"274":1},"Pclass":{"274":3},"Name":{"274":"Healy, Miss. Hanora \"Nora\""},"Sex":{"274":"female"},"Age":{"274":null},"SibSp":{"274":0},"Parch":{"274":0},"Ticket":{"274":"370375"},"Fare":{"274":7.75},"Cabin":{"274":null},"Embarked":{"274":"Q"}}


# Request a prediction from the server

In [22]:
pred_slug = 'get-pred?data={}&which_model=champion'.format(payload)
prediction_endpoint = endpoint_uri_base + pred_slug
response = requests.get(prediction_endpoint, verify=False, proxies={"https": endpoint_uri_base})
print(response.json())

{'prediction': 0, 'model_used': 'champ'}
